In [20]:
import re
from sys import prefix
import pandas as pd
import random
import os
import shutil
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt


class sudoku:
    def __init__(self, init=False, path='benchmarks', solver='sudoku_acs', execl='charts/base.xlsx'):
        self.path = path
        self.opt_path = path + '/INST_36x36/100'
        self.solver = solver
        self.data_path = 'results/' + solver
        self.records = pd.read_excel(execl, header=[0], index_col = [0])
        if init:
            self.records = self.records.drop(index=self.records.index)
            self.get_info(path)
            
    def get_benchmarks(self, percentage=0):
        file_list = os.listdir(self.opt_path)
        target_path = self.path + '/' + str(percentage)
        if not os.path.exists(target_path):
            os.makedirs(target_path)
        for file in tqdm(file_list, target_path):
            path = os.path.join(self.opt_path, file)
            with open(path) as f:
                grid = []
                order = f.readline()[:-1]
                is_sat = f.readline()
                square = int(order)*int(order)
                for i in range(square):
                    grid.append(f.readline().split())
                index = 0
                total = (100-percentage)*square*square/100
                while index < total:
                    x = random.randint(0, square-1)
                    y = random.randint(0, square-1)
                    if grid[x][y] != '-1':
                        grid[x][y] = '-1'
                        index += 1
                filename = file.split('_')
                filename[1] = str(percentage)
                filename = '_'.join(filename)
                with open(target_path + '/' + filename, 'w') as out:
                    out.write(order + '\n')
                    out.write(is_sat)
                    for i in range(square):
                        line = ''
                        for j in range(square):
                            line = line + grid[i][j] + '\t'
                        out.write(line[:-1] + '\n')

    def get_info(self, path):
        folder_list = os.listdir(path)
        for folder in folder_list:
            new_path = os.path.join(path, folder)
            if os.path.isdir(new_path):
                self.get_info(new_path)
            else:
                filename = new_path.split('/')[-1]
                category = filename.split('_')[0]
                percentage = filename.split('_')[1]
                self.records.loc[len(self.records.index)] = [filename, category, percentage]
        
    def save_to(self, filename):
        self.records.to_excel(filename)
        
    def arrange(self):
        # for filename in tqdm(folder_list, desc=folder_path):
        category_list = os.listdir(self.data_path)
        for folder in category_list:
            folder_path = os.path.join(self.data_path, folder)
            file_list = os.listdir(folder_path)
            for file in tqdm(file_list, desc=folder_path):
                path = os.path.join(folder_path, file)
                with open(path) as f:
                    try:
                        if self.solver == 'sudoku_acs':
                            while f.readline():
                                result = f.readline()[:-1]
                                if result == '0':
                                    result = 'sat'
                                elif result == '1':
                                    result = 'timeout'
                                time = f.readline()[:-1]
                                if result == 'timeout':
                                    time = '1000.000000'
                                info = f.readline()
                                # time = info.split(' : ')[-1][:-5]
                                name = info.split(' : ')[0].split('/')[-1]
                                # print(name)
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_lsc':
                            while f.readline():
                                info = f.readline()[:-1]
                                result = 'timeout'
                                time = '1000.000000'
                                name = info.split(' ')[0].split('/')[-1]
                                if 'find answer' in info:
                                    result = 'sat'
                                    time = info.split(': ')[-1]
                                    f.readline()
                                # print(name)
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                    except:
                        print(path)
    

In [16]:
# a = sudoku()
# for i in range(0, 100, 5):
    # a.get_benchmarks(percentage=i)

In [17]:
a = sudoku(init=True)
a.save_to('charts/base.xlsx')

In [21]:
b = sudoku(solver='sudoku_acs')
b.arrange()
b.save_to('charts/sudoku_acs.xlsx')

results/sudoku_acs/INST_9x9:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_acs/INST_36x36:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_acs/INST_16x16:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_acs/INST_25x25:   0%|          | 0/21 [00:00<?, ?it/s]

In [22]:
c = sudoku(solver='sudoku_lsc', execl='charts/sudoku_acs.xlsx')
c.arrange()
c.save_to('charts/sudoku.xlsx')

results/sudoku_lsc/INST_9x9:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_lsc/INST_36x36:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_lsc/INST_16x16:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_lsc/INST_25x25:   0%|          | 0/21 [00:00<?, ?it/s]